In [ ]:
from torchvision.datasets import MNIST
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

mnist_train = MNIST('./mnist', download=True)
mnist_text = MNIST('./mnist', download=True, train=False)

transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    transforms.Lambda(lambda x: torch.flatten(x))
    ])

# Create datasets for training & validation, download if necessary
training_set = MNIST('./mnist', train=True, transform=transform, download=True)
validation_set = MNIST('./mnist', train=False, transform=transform, download=True)

training_loader = torch.utils.data.DataLoader(training_set, batch_size=8, shuffle=True,
                                              num_workers=4, pin_memory=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=4, shuffle=False)

# PyTorch models inherit from torch.nn.Module
class MNISTClassifier(nn.Module):
    def __init__(self):
        super(MNISTClassifier, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(784, 128),
            nn.Sigmoid(),
            nn.Linear(128, 128),
            nn.Sigmoid(),
            nn.Linear(128, 10),
        )

    def forward(self, x):

        x = self.layers(x) # [batch_size, 784] --> [batch_size, 10]
        return x

model = MNISTClassifier()
model = model.cuda()

loss_fn = torch.nn.CrossEntropyLoss() # sum_i p_i logp_i
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    running_loss = 0.
    last_loss = 0.

    for i, (inputs, labels) in enumerate(training_loader):
        inputs, labels = inputs.cuda(), labels.cuda()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            avg_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, avg_loss))
            tb_x = epoch * len(training_loader) + i + 1
            writer.add_scalar('Loss/train', avg_loss, tb_x)
            running_loss = 0.

    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, (x, y) in enumerate(validation_loader):
            x, y = x.cuda(), y.cuda()
            outputs = model(x)
            loss = loss_fn(outputs, y)
            running_vloss += loss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

In [ ]:
!pip install lightning

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import lightning as pl
from torchmetrics import Accuracy

# Define the LightningModule
class MNISTClassifier(pl.LightningModule):
    def __init__(self, learning_rate=0.001):
        super(MNISTClassifier, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(784, 128),
            nn.Sigmoid(),
            nn.Linear(128, 128),
            nn.Sigmoid(),
            nn.Linear(128, 10),
        )
        self.loss_fn = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate
        self.val_logits = []
        self.val_labels = []
        self.val_accuracy = Accuracy('multiclass', num_classes=10)

    def forward(self, x):
        return self.layers(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)

        # Store logits and labels for later calculation
        self.val_logits.append(logits)
        self.val_labels.append(y)

    def on_validation_epoch_end(self):
        # Concatenate all stored logits and labels
        val_logits = torch.cat(self.val_logits, dim=0)
        val_labels = torch.cat(self.val_labels, dim=0)

        # Calculate the average loss over the entire validation set
        avg_val_loss = self.loss_fn(val_logits, val_labels)

        # Calculate accuracy using torchmetrics
        avg_val_acc = self.val_accuracy(val_logits, val_labels)

        # Log the average validation loss and accuracy
        self.log('val_loss', avg_val_loss, prog_bar=True)
        self.log('val_acc', avg_val_acc, prog_bar=True)

        # Clear the lists for the next epoch
        self.val_logits = []
        self.val_labels = []
        self.val_accuracy.reset()

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=self.learning_rate, momentum=0.9)

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger

# Data preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    transforms.Lambda(lambda x: torch.flatten(x))
])

train_dataset = MNIST('./mnist', train=True, transform=transform, download=True)
val_dataset = MNIST('./mnist', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=4, pin_memory=True)

# Instantiate the model
model = MNISTClassifier()

# Instantiate the Trainer
tb_logger = TensorBoardLogger(save_dir="logs/", name="cnn")

trainer = pl.Trainer(max_epochs=5,
                     accelerator="gpu",
                     devices=1,
                     log_every_n_steps=1000,
                     logger=tb_logger)

# Train the model
trainer.fit(model, train_loader, val_loader)

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger

class MNISTCNN(MNISTClassifier):
    def __init__(self, learning_rate=0.001):
        super(MNISTClassifier, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid(), ## nn.ReLu
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 10, kernel_size=1, stride=1, padding=0),
        )
        self.loss_fn = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate
        self.val_logits = []
        self.val_labels = []
        self.val_accuracy = Accuracy('multiclass', num_classes=10)

    def forward(self, x):
        x = self.layers(x)
        x = x.mean([2, 3])
        return x

# Data preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
#    transforms.Lambda(lambda x: torch.flatten(x))
])

train_dataset = MNIST('./mnist', train=True, transform=transform, download=True)
val_dataset = MNIST('./mnist', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=4, pin_memory=True)

# Instantiate the model
model = MNISTCNN(0.0001)

# Instantiate the Trainer
tb_logger = TensorBoardLogger(save_dir="logs/", name="cnn")

trainer = pl.Trainer(max_epochs=5,
                     accelerator="gpu",
                     devices=1,
                     log_every_n_steps=1000,
                     logger=tb_logger)

# Train the model
trainer.fit(model, train_loader, val_loader)

In [ ]:
#!pip install tensorboard
%load_ext tensorboard

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
%tensorboard --logdir logs/

In [ ]:
!rm logs -rf